<a href="https://colab.research.google.com/github/HSS107048212/KeywordsMap/blob/main/Example_KeywordsMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

安裝所需的Python套件：Pytrends、pandas和gspread-dataframe

In [17]:
!pip install Pytrends
!pip install pandas
!pip install gspread-dataframe==3.2.1

定義一個名為SPREADSHEET的變數，其值為Google試算表的URL。

In [18]:
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1CzhrUlWD8gLhvI9kBuuTGqWOuKpZpDSqf1kJEy_M62o/edit#gid=0'

匯入所需的Python套件和模組。

In [19]:
import pandas as pd
from pytrends.request import TrendReq
from google.colab import auth
import gspread
from google.auth import default
import time
import matplotlib.pyplot as plt
import seaborn as sns
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import urllib.parse
import pathlib

import contextlib

import requests
from pytrends.request import TrendReq

定義一個名為_TrendReq的類別，繼承自TrendReq類別，並定義GetGoogleCookie方法。

In [20]:
class _TrendReq(TrendReq):
    def GetGoogleCookie(self):
        # TODO: make sure to get rid of this dirty hack
        with _requests_get_as_post():
            return super().GetGoogleCookie()

定義一個名為_requests_get_as_post的上下文管理器，將requests.get方法替換為requests.post方法。

In [21]:
@contextlib.contextmanager
def _requests_get_as_post():
    requests.get, requests_get = requests.post, requests.get
    try:
        yield
    finally:
        requests.get = requests_get

使用Google帳號進行身份驗證，獲取驗證憑證，並使用該憑證進行Google試算表的授權。

In [22]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

記錄程式開始運行的時間，並建立一個_TrendReq類的實例pytrend，指定語言為英文（en-GB），時區為360。

In [23]:
startTime = time.time()
pytrend = _TrendReq(hl='en-GB', tz=360)

開啟Google試算表，獲取指定工作表（"Criteria"）的內容，並從工作表中讀取資料。df、wb和gc都指向同一個試算表。

In [24]:
df = wb = gc.open_by_url(SPREADSHEET)
sheet = wb.worksheet("Criteria")
df2 = sheet.col_values(5)
d_from = sheet.acell('B7').value
d_to = sheet.acell('B8').value

In [41]:
#測試
print('df2 是: {}'.format(df2))

df2 是: ['James', 'JAMES', 'Kobe', 'CURRY', 'Curry']


In [42]:
#測試
print('d_from 是: {}'.format(d_from))

d_from 是: 2022-05-19


In [43]:
#測試
print('d_to 是: {}'.format(d_to))

d_to 是: 2023-05-19


定義一個空列表dataset_time用於存儲資料集，設置一個延遲時間DELAY_TIME為2秒，定義另外三個空列表df_list、keywords和geos。

In [25]:
dataset_time = []
DELAY_TIME = 2
df_list = []
keywords = []


geos = list(sheet.col_values(7))

In [40]:
#測試之後 geos is a list.
print('geos 是: {}'.format(geos))

geos 是: ['GB', 'CN', 'TW']


遍歷geos列表中的元素，對每個地理位置進行相關搜尋。從df2中獲取關鍵字列表，使用pytrend.build_payload方法構建請求的參數，並進行相關搜尋。在處理過程中，如果出現錯誤，則延遲一段時間後再次嘗試。將獲得的資料加入dataset_time列表，並在資料中添加一個名為country的列。

將dataset_time中的每個字典資料轉換為pandas的DataFrame格式，並將其添加到datalist列表中。

將datalist中的每個DataFrame添加到df_list列表中。

In [26]:
for geo1 in geos:
  keywords = list(df2)

  pytrend.build_payload(kw_list=keywords, cat=0, timeframe= str(d_from + " " + d_to), geo= str(geo1))
  done = False
  while not done:
    try:
      data_time = pytrend.interest_over_time()
      done = True
      break
    except:
      time.sleep(DELAY_TIME) # delay because too many requests

  data_time['country']=geo1
  data_time = data_time.drop(labels=['isPartial'],axis='columns')
  dataset_time.append(data_time)


  datalist = []
  for i in dataset_time:
    result = pd.DataFrame(i)
    datalist.append(result)

  for i in datalist:
    df_list.append(i)

對df_list中的每個DataFrame進行處理，去除所有列中的缺失值，並將處理後的DataFrame添加到new_df_list列表中。

In [27]:
new_df_list = []

for i in df_list:
  new_i = i.dropna(axis=1, how='all')
  new_df_list.append(new_i)



對new_df_list中的每個DataFrame進行處理，過濾重複的資料，將不重複的DataFrame添加到final_df_list列表中。

In [28]:
final_df_list = None
for i in new_df_list:
  if final_df_list is None:
    final_df_list = [i]
  else:
    hasdf = False
    for j in final_df_list:
      if i.equals(j):
        hasdf = True
    if not hasdf:
      final_df_list.append(i)

對final_df_list中的每個DataFrame進行處理，重設索引，並將date列設置為新的索引，然後將處理後的DataFrame添加到new_final列表中。

In [29]:
new_final = []
for i in final_df_list:
  new_i = i.reset_index().set_index(['date'])
  new_final.append(new_i)

將new_final中的所有DataFrame合併為一個大的DataFrame，並將date列轉換為日期格式，然後將其設置為索引。

In [30]:
big_df = pd.concat(new_final).reset_index()
big_df['date'] = pd.to_datetime(big_df['date']).dt.date

big_df = big_df.set_index(['date'])

In [44]:
#輸出成果的dataframe
big_df

,James,JAMES,Kobe,CURRY,Curry,country
date,,,,,,
2022-05-22,60,58,1,16,17,GB
2022-05-29,57,57,1,17,17,GB
2022-06-05,57,57,1,17,17,GB
2022-06-12,58,59,1,18,18,GB
2022-06-19,57,57,1,17,16,GB
...,...,...,...,...,...,...
2023-04-16,0,0,0,0,0,TW
2023-04-23,0,0,0,0,0,TW
2023-04-30,79,79,48,100,100,TW


打開名為"Results"的工作表，清空其內容，並將big_df中的資料寫入該工作表。

In [31]:
wsResults = wb.worksheet("Results")
wsResults.clear()
set_with_dataframe(wsResults, big_df, include_index=True)